In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/randdense_u_1000_1/"

"../../graphs/randdense_u_1000_1/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [5]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [6]:
x = readFromFile(graphp * "x.vec");

In [7]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [8]:
numIts = 200;

@time for i in 4:5
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.659023 seconds (275.40 k allocations: 11.702 MB, 2.34% gc time)
  0.498645 seconds (123.72 k allocations: 4.980 MB)
  3.553850 seconds (3.15 M allocations: 95.011 MB, 2.17% gc time)
Working on iteration 10 with a-norm error 2370.4413133900402
Working on iteration 20 with a-norm error 1267.8490564041833
Working on iteration 30 with a-norm error 716.2441489893472
Working on iteration 40 with a-norm error 407.8763177339688
Working on iteration 50 with a-norm error 189.7620280981778
Working on iteration 60 with a-norm error 122.50175906019442
Working on iteration 70 with a-norm error 89.90791638496037
Working on iteration 80 with a-norm error 66.52248527622145
Working on iteration 90 with a-norm error 45.90831661898145
Working on iteration 100 with a-norm error 20.43113561973892
Working on iteration 110 with a-norm error 9.828327730479726
Working on iteration 120 with a-norm error 2.548524286068123
Working on iteration 130 with a-norm error 0.8718044803547098
Working on iteration 140 w